<table align="left"><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/ashishkrb7/Object-Detection-TF-data-preparation/blob/main/TF_trainer.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/ashishkrb7/Object-Detection-TF-data-preparation/blob/main/TF_trainer.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

In [ ]:
%reset -f

# Integrate Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd '/content/gdrive/MyDrive'

# Dependencies installation and environment setup

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
 
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
 
!pip install -q Cython contextlib2 pillow lxml matplotlib
 
!pip install -q pycocotools

!python -m pip install -q Cython pandas tf-slim lvis 

!python -m pip install -q tensorflow-addons

!python -m pip install -q tensorflow==1.14
 
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
 
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/MyDrive/models/research/:/content/gdrive/MyDrive/models/research/slim/'
 
!python object_detection/builders/model_builder_test.py

# Remove unnecessary files and folder from the directory

In [ ]:
# import shutil,os
# shutil.rmtree("/content/gdrive/MyDrive/Tensorflow/models")
# os.remove("/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/exporter_main_v2.py")

# cd into the TensorFlow directory in your Google Drive and download models

        TensorFlow
        ├───scripts
        │   └───preprocessing
        │     └───generate_tfrecord.py 
        │     └───xml_to_csv.py 
        └───workspace
            └───training_demo
                ├───annotations
                │   └───label_map.pbtxt 
                ├───exported-models
                ├───images
                │   ├───test
                │   │     └───test images with corresponding XML files (put zip file, code will do rest of the work)
                │   └───train
                │         └───train images with corresponding XML files (put zip file, code will do rest of the work)
                ├───models
                │   └───my_ssd_resnet50_v1_fpn
                │     └───pipeline.config
                └───pre-trained-models
                    └───ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

In [ ]:
%cd '/content/gdrive/MyDrive'

In [1]:
repo_dir_path='.'

In [2]:
# create directory
if not os.path.exists(repo_dir_path+'/Tensorflow/scripts/preprocessing'):os.makedirs(repo_dir_path+'/Tensorflow/scripts/preprocessing')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/annotation'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/annotation')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/exported-models'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/exported-models')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/images/train'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/images/train')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/images/test'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/images/test')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/models'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/trainingdemo/models')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/pre-trained-models'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/pre-trained-models')

# Load files

In [ ]:
# Load TF-recorder generator
import urllib.request
url_generate_tfrecord = "https://raw.githubusercontent.com/ashishkrb7/Object-Detection-TF-data-preparation/main/generate_tfrecord.py"
url_xml_to_csv = "https://raw.githubusercontent.com/ashishkrb7/Object-Detection-TF-data-preparation/main/generate_tfrecord.py"
filename, headers = urllib.request.urlretrieve(url_generate_tfrecord, filename=repo_dir_path+"/Tensorflow/scripts/preprocessing/generate_tfrecord.py")
filename, headers = urllib.request.urlretrieve(url_xml_to_csv, filename=repo_dir_path+"/Tensorflow/scripts/preprocessing/xml_to_csv.py")

In [ ]:
# Load train and test dataset in zip formate to respective folder
from google.colab import files
%cd {repo_dir_path+'/Tensorflow/workspace/training_demo/images/train'}
upload=files.upload()

In [ ]:
!unzip train.zip
!rm train.zip

In [ ]:
%cd {repo_dir_path+'/Tensorflow/workspace/training_demo/images/test'}
upload=files.upload()

In [ ]:
!unzip test.zip
!rm test.zip

# Download tensorflow Object detection API

In [ ]:
%cd '/content/gdrive/MyDrive/Tensorflow
!git clone https://github.com/tensorflow/models.git
!git pull

# Dataset preparation (TFrecord preparation)

In [ ]:
%cd './content'

In [ ]:
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python gdrive/MyDrive/Tensorflow/scripts/preprocessing/xml_to_csv.py -i gdrive/MyDrive/Tensorflow/workspace/training_demo/images/train -o gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/train_labels.csv -l gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation
 
# Convert test folder annotation xml files to a single csv.
!python gdrive/MyDrive/Tensorflow/scripts/preprocessing/xml_to_csv.py -i gdrive/MyDrive/Tensorflow/workspace/training_demo/images/test -o gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/test_labels.csv
 
# Generate `train.record`
!python gdrive/MyDrive/Tensorflow/scripts/preprocessing/generate_tfrecord.py --csv_input=gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/train_labels.csv --output_path=gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/train.record --img_path=gdrive/MyDrive/Tensorflow/workspace/training_demo/images/train --label_map gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/label_map.pbtxt
 
# Generate `test.record`
!python gdrive/MyDrive/Tensorflow/scripts/preprocessing/generate_tfrecord.py --csv_input=gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/test_labels.csv --output_path=gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/test.record --img_path=gdrive/MyDrive/Tensorflow/workspace/training_demo/images/train --label_map gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/label_map.pbtxt


In [ ]:
inputDir='/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/'
%cd {inputDir}
!zip -r data.zip {inputDir}

In [ ]:
from google.colab import files
files.download('data.zip')

# Download pre-trained model

To select the model for TF-1 [click here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md)

Reference:
-   [TF1](https://towardsdatascience.com/object-detection-by-tensorflow-1-x-5a8cb72c1c4b)
-   [TF2](https://medium.com/swlh/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc)

In [ ]:
model_name='ssd_inception_v2_coco_2018_01_28'

In [ ]:
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/models/{model_name}'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/models/{model_name}')

In [ ]:
import wget
fname=wget.download(f'http://download.tensorflow.org/models/object_detection/{model_name}.tar.gz')
import shutil
shutil.unpack_archive(fname,repo_dir_path+'/Tensorflow/workspace/training_demo/models')
!rm {fname}

In [ ]:
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/pre-trained-models/{model_name}'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/pre-trained-models/{model_name}')

In [ ]:
# Copy pipline.config file. This file need to modify based on dataset
shutil.copy(repo_dir_path+'/Tensorflow/workspace/training_demo/pre-trained-models/{model_name}/pipline.config',repo_dir_path+'/Tensorflow/workspace/training_demo/model/{model_name}')

# Configuring a Training Pipeline

Based on research, Following parameter are found to be sensitive